In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Conv2d, Linear
from torch.nn import Dropout
from torch.nn import MaxPool2d, AvgPool2d
from torch.nn import ReLU
from torch.nn.modules.batchnorm import BatchNorm1d, BatchNorm2d
from torch.nn import LogSoftmax, Sigmoid
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageSequence
from torchvision.io import read_image
from torch import flatten

In [ ]:
#creating new folder for png images
if not os.path.exists('png'):
    os.makedirs('png')

In [ ]:
channel = 2 #write here the number of the GFP channel
measurementFileName='measurements3.csv' #write here the name of the measurement file

In [ ]:
centroids = pd.DataFrame()
#iterating over the images
for i in range(2, 3):
    for j in range(1,4):
        try:
            #loading the csv file of the corresponding image
            data = pd.read_csv(f'cells/Image_{i:03}_0{j}-centroids.csv', header=None)
            centroids = pd.concat([centroids, data], ignore_index=True)
            #converting each image to png
            for k in range(0, len(data)):
                with Image.open('cells/' + data.iloc[k, 4] + '.tif') as im:
                    tmp = np.asarray(ImageSequence.all_frames(im)[channel-1], dtype=np.uint8)[:64, :64]
                    im2 = Image.fromarray(tmp)
                    im2.save('png/'+ data.iloc[k, 4] + '.png')
            print(f'Image_{i:03}_0{j} done')
        except pd.errors.EmptyDataError:
            print(f'Image_{i:03}_0{j} not found')
        except FileNotFoundError:
            print(f'Image_{i:03}_0{j} not found')

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 4]+'.png')
        image = read_image(img_path)
        
        image = image.to(torch.float32)
        
        return image

In [ ]:
class LeNet(nn.Module):
    def __init__(self, numChannels, hidensize, classes):
        super(LeNet, self).__init__()
        
        # Dropout
        self.dropout = nn.Dropout(0.25)
        self.relu = ReLU(inplace=True)
        self.norm0 = BatchNorm2d(1)

        # Layer 1
        self.conv1 = Conv2d(in_channels=numChannels, out_channels=30, kernel_size=(5, 5))
        self.norm1 = BatchNorm2d(30)
        self.pool1avg = AvgPool2d(kernel_size=4, stride=1)
        self.pool1max = MaxPool2d(kernel_size=2, stride=1)

        # Layer 2
        self.conv2 = Conv2d(in_channels=30, out_channels=40, kernel_size=(5, 5))
        self.norm2 = BatchNorm2d(40)
        self.pool2max = MaxPool2d(kernel_size=2, stride=2)

        # Layer 3
        self.conv3 = Conv2d(in_channels=40, out_channels=50, kernel_size=(5, 5))
        self.norm3 = BatchNorm2d(50)
        self.pool3max = MaxPool2d(kernel_size=2, stride=2)

        # Layer 4
        self.conv4 = Conv2d(in_channels=50, out_channels=60, kernel_size=(5, 5))
        self.norm4 = BatchNorm2d(60)
        self.pool4max = MaxPool2d(kernel_size=2, stride=2)

        # Classification
        self.fc1 = Linear(in_features=540, out_features=hidensize)
        self.fc2 = Linear(in_features=hidensize, out_features=hidensize)
        self.fc3 = Linear(in_features=hidensize, out_features=hidensize)
        self.fc4 = Linear(in_features=hidensize, out_features=classes)
        self.activation = LogSoftmax(dim=1)

    def forward(self, x):
        #x = self.norm0(x)
        
        # First Layer
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu(x)
        x = self.pool1avg(x)
        x = self.pool1max(x)
        # Second Layer
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu(x)
        x = self.pool2max(x)
        # Third Layer
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu(x)
        x = self.pool3max(x)
        # Fourth Layer
        x = self.conv4(x)
        x = self.norm4(x)
        x = self.relu(x)
        x = self.pool4max(x)
        # Linear
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x) # Dropouts avoid overfit
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.dropout(x)
        # Classifier
        output = self.activation(x)
        return output

In [ ]:
#loading the model
modl = LeNet(1, 500, 2)
path = 'model.zip'
if not torch.cuda.is_available():
    print('No GPU found. Please use a GPU to train your neural network.')
modl.load_state_dict(torch.load(path))

#creating the dataset with the images you want to classify
test = CustomImageDataset(centroids, './png')
testldr = DataLoader(dataset=test, batch_size=20, shuffle=False)
modl.eval()

#predicting the labels
labels = np.array([])
for img in testldr:
    with torch.no_grad():
        testoutp = modl(img)
        pred = torch.max(testoutp, 1)[1].data.squeeze()
        labels = np.append(labels, pred.numpy())

#adding the labels to the dataframe
centroids.iloc[:,1]=labels

In [ ]:
centroids

In [ ]:
#merging the predicted labels with the measurements
measures = pd.read_csv(measurementFileName, sep=',')
centroids.iloc[:,2] = [round(i, 1) for i in centroids.iloc[:,2]]
centroids.iloc[:,3] = [round(i, 1) for i in centroids.iloc[:,3]]
new_meas = measures.merge(centroids, left_on=['Image', 'Centroid X µm', 'Centroid Y µm'], right_on=[0, 2, 3], how='left').drop(columns=[0, 2, 3, 4]).rename(columns={1: 'Prediction'})
print(f'{sum(new_meas["Prediction"].isna())-sum(measures["Nucleus: Area µm^2"]<50)} neurons could not find a prediction')

In [ ]:
#saving the measurements with the predicted labels!
new_meas.to_csv('measurements4.csv', index=False)